In [1]:
import requests
from tqdm import tqdm_notebook

In [2]:
%%time

#id / age / comment / rate리스트 선언
id_list = [] #댓글별 id번호
age_list = [] #연령대
comment_list = [] #댓글내용
rate_list = [] #평점
review_total_list = [] #리뷰에 대한 피드백 전체
review_likes_list = [] #리뷰를 좋아한 피드백


for page in tqdm_notebook(range(1,2000)): #총 게시물 현재 8,615개 -> 여유있게 10000개까지

    url = "http://widgets4.cre.ma/bodyluv.kr/products/reviews?product_code=52&iframe_id=crema-product-reviews-2&app=0&parent_url=http%3A%2F%2Fwww.bodyluv.kr%2Fproduct%2F%25EB%25A7%2588%25EC%2595%25BD%25EB%25A7%25A4%25ED%258A%25B8%25EB%25A6%25AC%25EC%258A%25A4%2F52%2Fcategory%2F25%2Fdisplay%2F1%2F%3Fcrema-product-reviews-2-page%3D2&page=" + str(page) + "&iframe=1"
    

    text = requests.get(url).text
    
    
    id_r = text.split("<li\n  id=") #id는 별도로 가지고 있음.
    
    r = text.split("<div class=\"products_reviews_list_review__message\">") #age, comment 둘다 가지고 있는 html
    
    rate_r = text.split("<div class=\"products_reviews_list_review__score_text_rating\">")
    
    review_r = text.split("<div class=\"products_reviews_list_review__score\">")
        
    if len(r) == 1:
        break
        
        
    for x in id_r[1:]:
        
        id = x.split("\n")[0]
        id = id.replace('review_', "").replace('"',"")
        id_list.append(id)
        
    for y in r[1:]:
        
        if "<div class=\"products_reviews_list_review__message_content\">" in y:
            
            comment = y.split("<div class=\"products_reviews_list_review__message_content\">")[1].split("\n\n            </div>")[0]
            comment = comment.replace("\n              ", "").replace("<br>", " ")
            
            if "연령대</div>" in y:
                age = y.split("연령대</div>")[1].split("</div>")[0]
                age = age.replace("<div class=\"review_option__content\">", "").replace("\n        ","")
            else:
                age = "None"
            
            age_list.append(age)
            comment_list.append(comment)
            
        else:
            pass
    
    for z in rate_r[1:]:
        
        rate = z.split("</div>")[0]
        rate = rate.replace('- ', "")
        rate_list.append(rate)
        
    for k in review_r[1:]:
        
        #if문이 없어도 아래의 split의 text가 없으면 0을 출력   
        review_total = k.split("<strong class=\'js-like-score-total\'>")[1].split("</strong>")[0]
        review_likes = k.split("<strong class=\'js-like-score-plus\'>")[1].split("</strong>")[0]

        review_total_list.append(review_total)
        review_likes_list.append(review_likes)


CPU times: user 11.5 s, sys: 1.35 s, total: 12.9 s
Wall time: 27min 4s


In [3]:
print(len(id_list), len(age_list), len(comment_list), len(rate_list), len(review_total_list), len(review_likes_list))  # 전체 데이터 길이 확인.

8468 8468 8468 8468 8468 8468


In [4]:
#데이터 프레임 작성

In [5]:
data = {'id': id_list,
        'age': age_list,
        'rate' : rate_list,
        'review_total' : review_total_list,
        'review_likes' : review_likes_list,
        'comment': comment_list}

In [6]:
df = pd.DataFrame(data, columns=["id", "age", "rate", "review_total", "review_likes", "comment"])
df.tail()

,id,age,rate,review_total,review_likes,comment
8463,62933,None,아주 좋아요,1,0,불만족 별루에서 다 눌려서오고 비추 (2018-03-04 11:28:12 에 등록...
8464,62876,30대,그냥 그래요,1,0,솔직히.. 과대 광고네요
8465,60546,None,아주 좋아요,1,0,보통 방수 커버 씌우다 죽는줄 ㅋㅋㅋㅋ 커버 원단과 매트 제질이 상극 이네요 (2...
8466,60547,None,아주 좋아요,1,0,보통 냄새가 너무 심해요. 진짜 너무 심함 그리고 되게 잘 찢어지네요 조심들 하세요...
8467,57991,None,아주 좋아요,1,0,"불만족 처음 꺼낼때 냄새가심함, 지버분함,모서리는 눌려온게 복구가안됨0 (2018..."


In [8]:
df.shape

(8468, 6)

In [9]:
df.to_csv('mattress_result')